Будем исследовать пространство из 4 признаков:  
a1_mean , a2_mean , a3_mean , a4_mean

In [1]:
import numpy as np
from matplotlib import pyplot as plt
 
import pandas as pd


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix,ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler ,MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
 

In [ ]:
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from joblib import dump, load 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from sklearn.base import BaseEstimator
from keras.utils import to_categorical

In [3]:
from joblib import dump, load

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Подготовка вспомогательных функций

In [ ]:
def fit_and_choose(X,Y , model ,  grid_par , score_mes , spliter_cvd , verbs ,n_proc ):
  gsC =GridSearchCV(estimator= model,
             param_grid=   grid_par,
             scoring =score_mes,
             refit=True,
             cv = spliter_cvd ,
             verbose =  verbs,
             n_jobs = n_proc
             )
  gsC.fit(X, Y)
  return {'best model': gsC.best_estimator_ ,
          'best params' : gsC.best_params_,
          'best score': gsC.best_score_}
   

In [ ]:
def conf_matr_and_score_f1(Y_predicted , Y_true,le):

  cmatr  = confusion_matrix(le.inverse_transform(Y_true),le.inverse_transform(Y_predicted) , labels=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
  score_f1 = f1_score(Y_true,Y_predicted  , average='micro')
  cmatr_df = pd.DataFrame(cmatr,columns=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'], index=['sitting', 'sittingdown', 'standing', 'standingup', 'walking'])
  return {'conf_matr': cmatr_df , 'f1-score':score_f1 }

In [5]:
def train_val_test_split(X,Y, rand_state):
  X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.4, random_state=rand_state  ,stratify = Y )
  X_val, X_test, Y_val, Y_test = train_test_split(X_val,  Y_val, test_size=0.5, random_state=rand_state ,stratify =  Y_val )
    
  return X_train, X_val,X_test,Y_train, Y_val, Y_test

In [ ]:
def scale_x(X1,X2,X3, type_of_scale = None):
  if type_of_scale == 'minmax':
    scaler = MinMaxScaler()
  else: scaler = StandardScaler()
  scaler = scaler.fit(X1)
  X1  = scaler.transform( X1 )
  X2 =  scaler.transform( X2 )
  X3 =  scaler.transform( X3 )
  return X1, X2, X3

Загрузка данных из файла

In [6]:
data = pd.read_csv('/content/drive/MyDrive/данные с акселерометра/dataset_w8_o25.csv', delimiter = ';')

Кодирование признаков и забор столбцов

In [7]:
le =  LabelEncoder()
le.fit(data['class'])
Y = le.transform(data['class'])
 # preparing X data and split
X =np.array( data[['a1_mean', 'a2_mean',  'a3_mean', 'a4_mean']]) 
 

Разделение выборки на треровочную , валидационную  и тестовую 60 20 20

In [8]:
X_train, X_val, X_test,  Y_train, Y_val,Y_test = train_val_test_split(X, Y, rand_state=42 )
 

Масштабирование по стандартизации

In [ ]:
X_train_std , X_val_std , X_test_std = scale_x(X_train, X_val, X_test)


Масштабирование по минимаксу

In [ ]:
X_train_mn , X_val_mn , X_test_mn = scale_x(X_train, X_val, X_test, 'minmax')

Оценка softmax регрессии

In [ ]:
 C_pars = [i*0.25 for i in range(1,60)]
 reslt_softmax = fit_and_choose(X_train_std ,Y_train , 
                        LogisticRegression( penalty ='l2',multi_class='multinomial',solver='newton-cg',verbose=2) , 
                       {'C':  C_pars},
                       'f1_micro' , 
                        StratifiedKFold(n_splits=5,  shuffle=True, random_state = 42 ), 4,-1  )

Fitting 5 folds for each of 59 candidates, totalling 295 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 295 out of 295 | elapsed:  5.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.6s finished


In [ ]:
 reslt_softmax 

{'best model': LogisticRegression(C=11.25, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='multinomial', n_jobs=None, penalty='l2',
                    random_state=None, solver='newton-cg', tol=0.0001, verbose=2,
                    warm_start=False),
 'best params': {'C': 11.25},
 'best score': 0.7933294494611128}

Оценка на валидационном наборе

In [ ]:
Y_pred_softmax = reslt_softmax['best model'].predict(X_val_std)
result_of_softmax = conf_matr_and_score_f1(Y_pred_softmax ,Y_val , le) 

In [ ]:
result_of_softmax['conf_matr']

,sitting,sittingdown,standing,standingup,walking
sitting,3491,67,2,0,2
sittingdown,60,614,96,41,21
standing,96,0,2622,121,449
standingup,1,151,200,467,9
walking,0,0,981,0,1953


In [ ]:
result_of_softmax['f1-score']

0.799283467319119

Оценка решающего дерева

In [ ]:
tree_params = {'criterion': [ 'gini', 'entropy' ],
               'max_depth' : [6,8,10,12, 14, 16, 18,20],
               'min_samples_split': [30,40,50,60,70 ] ,
               'min_samples_leaf' : [2,10,20,30,40],
              'random_state': [48,56,78,91,112,134 ],
               'max_features': [2,3,4]}

In [ ]:
reslt_tree = fit_and_choose(X_train_std,Y_train , 
                       DecisionTreeClassifier( splitter = 'random'), 
                       tree_params,
                       'f1_micro' , 
                        StratifiedKFold(n_splits=5,  shuffle=True, random_state = 42 ), 4,-1  )

Fitting 5 folds for each of 7200 candidates, totalling 36000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 348 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 1184 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 2552 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 4320 tasks      | elapsed:   48.1s
[Parallel(n_jobs=-1)]: Done 6472 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 9024 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 11960 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 15296 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 19016 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 23136 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 27640 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 32544 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 36000 out of 36000 | elapsed:  7.3min finished


In [ ]:
reslt_tree

{'best model': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                        max_depth=20, max_features=4, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=2, min_samples_split=30,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=91, splitter='random'),
 'best params': {'criterion': 'entropy',
  'max_depth': 20,
  'max_features': 4,
  'min_samples_leaf': 2,
  'min_samples_split': 30,
  'random_state': 91},
 'best score': 0.9826682202155549}

Оценка на валидационном наборе

In [ ]:
Y_pred_tree = reslt_tree['best model'].predict(X_val_std)
result_of_tree  = conf_matr_and_score_f1(Y_pred_tree ,Y_val , le)   

In [ ]:
result_of_tree['conf_matr'] 

,sitting,sittingdown,standing,standingup,walking
sitting,3558,2,0,2,0
sittingdown,9,781,6,23,13
standing,0,11,3227,3,47
standingup,0,21,0,807,0
walking,0,14,59,1,2860


In [ ]:
result_of_tree['f1-score']

0.9815623907724572

Оценка метода ближайших соседей без взвешенной метрики

In [ ]:
knn_params1 = {'p': [2,3,4] , 'n_neighbors': [i for i in range(2,21) ]}

In [ ]:
 reslt_knn1 = fit_and_choose(X_train_mn , Y_train , 
                        KNeighborsClassifier( weights= 'distance' ,  metric = 'minkowski',  ) , 
                       knn_params1,
                       'f1_micro' , 
                        StratifiedKFold(n_splits=5,  shuffle=True, random_state = 42 ), 4,-1  )

Fitting 5 folds for each of 57 candidates, totalling 285 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 118 tasks      | elapsed:   31.2s
[Parallel(n_jobs=-1)]: Done 241 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 285 out of 285 | elapsed:  1.6min finished


In [ ]:
 reslt_knn1 

{'best model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                      weights='distance'),
 'best params': {'n_neighbors': 2, 'p': 2},
 'best score': 0.9948732886688028}

Оценка на валидационном наборе

In [ ]:
Y_pred_knn1 = reslt_knn1['best model'].predict(X_val_mn)
result_of_knn1  = conf_matr_and_score_f1(Y_pred_knn1 ,Y_val , le) 

In [ ]:
result_of_knn1['conf_matr'] 

,sitting,sittingdown,standing,standingup,walking
sitting,3562,0,0,0,0
sittingdown,3,812,4,9,4
standing,0,0,3274,0,14
standingup,0,1,0,827,0
walking,0,0,22,0,2912


In [ ]:
result_of_knn1['f1-score']

0.9950192240475358

Оценка метода ближайших соседей со взвешенной метрикой

In [ ]:
 size_of_vars_w = 25
 met_weights = np.random.randint(10,300 ,size = (size_of_vars_w ,4))/100
 met_pars = [{'w' : met_weights[i] } for i in range(size_of_vars_w )] 

In [ ]:
met_pars = [{'w': np.array([1.14, 1.18, 1.95, 2.85])},
 {'w': np.array([1.96, 1.85, 0.81, 1.92])},
 {'w': np.array([2.73, 0.72, 2.64, 0.3 ])},
 {'w': np.array([1.32, 1.33, 0.16, 0.82])},
 {'w': np.array([0.17, 0.37, 1.32, 0.81])},
 {'w': np.array([2.59, 2.5 , 0.61, 1.18])},
 {'w': np.array([0.81, 1.68, 1.31, 1.38])},
 {'w': np.array([0.89, 2.65, 0.97, 1.6 ])},
 {'w': np.array([0.97, 2.62, 2.43, 1.48])},
 {'w': np.array([1.85, 2.14, 2.82, 1.9 ])},
 {'w': np.array([0.52, 1.27, 1.06, 0.39])},
 {'w': np.array([0.16, 2.66, 1.56, 2.26])},
 {'w': np.array([1.43, 2.72, 1.3 , 1.53])},
 {'w': np.array([2.74, 2.14, 2.32, 0.45])},
 {'w': np.array([1.11, 0.39, 2.13, 2.88])},
 {'w': np.array([2.38, 2.86, 2.36, 0.73])},
 {'w': np.array([0.17, 2.02, 1.08, 1.2 ])},
 {'w': np.array([1.7 , 1.53, 2.82, 0.86])},
 {'w': np.array([1.67, 1.64, 0.37, 0.95])},
 {'w': np.array([0.81, 0.3 , 0.35, 2.59])},
 {'w': np.array([2.76, 0.86, 1.36, 2.13])},
 {'w': np.array([0.83, 2.47, 1.9 , 1.78])},
 {'w': np.array([1.2 , 0.97, 1.15, 0.66])},
 {'w': np.array([0.2 , 2.43, 0.76, 2.1 ])},
 {'w': np.array([0.49, 0.77, 1.44, 2.8 ])}]

In [ ]:
reslt_knn2 = fit_and_choose(X_train_mn,Y_train , 
                        KNeighborsClassifier(n_neighbors = 2, weights= 'distance' ,p =2 ,  metric = 'wminkowski',  ) , 
                       {   'metric_params' : met_pars   },
                       'f1_micro' , 
                        StratifiedKFold(n_splits=5,  shuffle=True, random_state = 42 ), 4,-1  )

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:  9.5min finished


In [ ]:
reslt_knn2

{'best model': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='wminkowski',
                      metric_params={'w': array([2.38, 2.86, 2.36, 0.73])},
                      n_jobs=None, n_neighbors=2, p=2, weights='distance'),
 'best params': {'metric_params': {'w': array([2.38, 2.86, 2.36, 0.73])}},
 'best score': 0.9964753859598019}

Оценка на валидационном наборе

In [ ]:
Y_pred_knn2 = reslt_knn2['best model'].predict(X_val_mn)
result_of_knn2  = conf_matr_and_score_f1(Y_pred_knn2 ,Y_val , le)  

In [ ]:
result_of_knn2['conf_matr'] 

,sitting,sittingdown,standing,standingup,walking
sitting,3561,0,1,0,0
sittingdown,3,816,2,7,4
standing,0,0,3281,1,6
standingup,0,0,1,827,0
walking,0,0,11,0,2923


In [ ]:
result_of_knn2['f1-score']

0.9968542467668647

Оценка нейронной сети

Класс - нейронная сеть

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from sklearn.base import BaseEstimator
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.models import load_model
#from  keras  import save 
#from keras.callbacks import EarlyStopping

#, n_inputs , n_outputs , neurons_in_hiden,
 #                         activation , optimizer_type , metric , dropout_pr ,tol
class Keras_mlpclassifier( BaseEstimator  ):
    # constructor for network
    # loss - categorial cross entropy
    # activation on output layer - softmax
    
    def __init__(self, n_inputs , n_outputs , neurons_in_hiden, activation , optimizer_type , metrics , dropout_pr ,tol  ):
		   # n_inputs , n_outputs , neurons_in_hiden, activation , optimizer_type , metrics , dropout_pr ,tol  
       
       self.n_inputs = n_inputs  
       self.n_outputs = n_outputs
       self.neurons_in_hiden = neurons_in_hiden
       self.activation = activation
       self.optimizer_type = optimizer_type
        
       self.metrics = metrics
       self.dropout_pr = dropout_pr
       self.tol = tol
        
       self.ann = Sequential()

       N_layers = len(neurons_in_hiden) 
       self.ann.add(Dense(neurons_in_hiden[0] , input_dim= n_inputs  , activation = activation[0]) )
       self.ann.add(Dropout(dropout_pr))

       for i in range(1,N_layers):
           
          self.ann.add(Dense(neurons_in_hiden[i] , input_dim= n_inputs  , activation = activation[i]) )
          self.ann.add(Dropout(dropout_pr))

       self.ann.add(Dense(n_outputs ,  activation = 'softmax'))
 
       self.ann.compile(loss='categorical_crossentropy', optimizer=optimizer_type, metrics = metrics ) 
              
    
    def fit(self, X, Y, epochs, batch_size , verb):

      self.ann.fit( X,to_categorical(Y)  , epochs = epochs, batch_size = batch_size, verbose =verb )


      


    def predict_proba(self, X ):
        answers_prob = self.ann.predict(X)
        return answers_prob 

    def predict(self, X ):
        answers = np.argmax(self.predict_proba(X) , axis=1)
        return answers
    
    def save(self , path):
        self.ann.save(path)
   
    def load(self , path):
        self.ann = load_model(path)

Оверсемплинг обучающей выборки

In [ ]:
kolvo_unique = np.unique(Y_train,return_counts= True)[1]
delta = np.array([np.max(kolvo_unique) for i in range(5)]) - kolvo_unique
X_1 = X_train_mn[Y_train == 1]
X_2 = X_train_mn[Y_train == 2]
X_3 = X_train_mn[Y_train == 3]
X_4 = X_train_mn[Y_train == 4] 
generator = np.random.seed(10000)

kort_mass = [X_1, X_2 , X_3 ,X_4]
kort_mass_copies = [X_1, X_2 , X_3 ,X_4]
Y_tr_add = np.array([])
for i in range(4):
  lengthm = kort_mass[i].shape[0]
  nd_copy = (10685 // lengthm) -1
  nd_choise = 10685 % lengthm
  if nd_copy != 0 :
    for t in range(nd_copy):
      kort_mass_copies[i] = np.vstack(( kort_mass_copies[i] , kort_mass[i]  ))
  kort_mass_copies[i]  = np.vstack(( kort_mass_copies[i] ,  
                                      kort_mass[i][-nd_choise:]  ))  
  Y_tr_add = np.hstack((Y_tr_add , np.array([i+1 for t in range(10685) ] )  ) )
X_train2 = np.vstack(kort_mass_copies)

X_train2 = np.vstack(( X_train_mn[Y_train == 0] , X_train2)) 
Y_tr_add = np.hstack(( Y_train[Y_train == 0] , Y_tr_add    ) )

In [ ]:
index_sh = np.arange(Y_tr_add.shape[0])
np.random.shuffle(index_sh  ) 

Модели - кандидаты для отбора

In [ ]:
ann_models =[ Keras_mlpclassifier(4 , 5 , [3,2] , [ 'relu','sigmoid'] , SGD(learning_rate=0.007 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 ) ,
              Keras_mlpclassifier(4 , 5 , [3 ] , [ 'relu'  ] , SGD(learning_rate=0.007 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 )  ,
              Keras_mlpclassifier(4 , 5 , [ 2] , [ 'relu' ] , SGD(learning_rate=0.007 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 ) ,
             
              Keras_mlpclassifier(4 , 5 , [3 ] , [ 'relu' ] , 'Adam' ,[ 'CategoricalAccuracy'] , 0.4,0.00001 ) ,
             Keras_mlpclassifier(4 , 5 , [3 , 2 ] , [ 'relu','sigmoid' ] , 'Adam' ,[ 'CategoricalAccuracy'] , 0.4,0.00001 )  ,
             Keras_mlpclassifier(4 , 5 , [ 2] , [ 'relu'] ,'Adam',[ 'CategoricalAccuracy'] , 0.4,0.00001 ) ,
            
              Keras_mlpclassifier(4 , 5 , [3 ] , [  'sigmoid'] , SGD(learning_rate=0.01 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 )  ,
              Keras_mlpclassifier(4 , 5 , [ 2] , [  'sigmoid'] , SGD(learning_rate=0.01 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 ) ,]

In [ ]:
list_of_f1_ann = np.zeros(len(ann_models ))
list_of_c_matr_ann = []

In [ ]:
import tensorflow as tf

Обучение и проверка на валидационном наборе

In [ ]:
with tf.device('/CPU:0'):
  for i in range(len(ann_models )):
    print('train model ',i+1)
    print(ann_models[i])
    ann_models[i].fit(X_train2[index_sh ] , Y_tr_add[index_sh ]  , 1800 , 200 ,1 )
    print('estimation')
    Y_ann = ann_models[i].predict(X_val_mn)
    res_mod_i = conf_matr_and_score_f1(Y_ann ,Y_val , le)
    list_of_f1_ann[i] = res_mod_i['f1-score']
    list_of_c_matr_ann.append(res_mod_i['conf_matr'])

Выходные данные были обрезаны до нескольких последних строк (5000).
268/268 [==============================] - 1s 4ms/step - loss: 1.3559 - categorical_accuracy: 0.3752
Epoch 1107/1800
268/268 [==============================] - 1s 5ms/step - loss: 1.3520 - categorical_accuracy: 0.3823
Epoch 1108/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.3464 - categorical_accuracy: 0.3879
Epoch 1109/1800
268/268 [==============================] - 1s 5ms/step - loss: 1.3473 - categorical_accuracy: 0.3798
Epoch 1110/1800
268/268 [==============================] - 1s 5ms/step - loss: 1.3407 - categorical_accuracy: 0.3895
Epoch 1111/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.3478 - categorical_accuracy: 0.3820
Epoch 1112/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.3519 - categorical_accuracy: 0.3832
Epoch 1113/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.3490 - categorical_accuracy: 0.3769
Epoch 1114/1

ValueError: ignored

In [ ]:
list_of_f1_ann

array([0.58310031, 0.84026564, 0.5661482 , 0.71102761, 0.4440755 ,
       0.        , 0.        , 0.        ])

In [ ]:
np.argmax(list_of_f1_ann)

1

In [ ]:
list_of_c_matr_ann[1]

,sitting,sittingdown,standing,standingup,walking
sitting,3501,42,19,0,0
sittingdown,51,629,20,127,5
standing,15,7,2245,113,908
standingup,0,193,3,632,0
walking,0,6,284,35,2609


Лучшая нейросеть
4 , 3 ,5
relu 
 

In [ ]:
ann_list = [Keras_mlpclassifier(4 , 5 , [3 ] , [ 'relu'  ] , SGD(learning_rate=0.007 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 ),
            Keras_mlpclassifier(4 , 5 , [3 ] , [ 'relu'  ] , SGD(learning_rate=0.01 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 ),
            Keras_mlpclassifier(4 , 5 , [3 ] , [ 'sigmoid'  ] , SGD(learning_rate=0.007 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 ),
            Keras_mlpclassifier(4 , 5 , [3 ] , [ 'sigmoid'  ] , SGD(learning_rate=0.01 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 ) ]

In [ ]:
ann = Keras_mlpclassifier(4 , 5 , [3 ] , [ 'relu'  ] , SGD(learning_rate=0.01 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 )

In [ ]:
ann.save('/content/drive/MyDrive/models_for_har4/ann.h5')

In [ ]:
ann.load('/content/drive/MyDrive/models_for_har4/ann.h5')

In [ ]:
Y_ann = ann.predict(X_val_mn)
res_mod_i = conf_matr_and_score_f1(Y_ann ,Y_val , le)

In [ ]:
res_mod_i

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting            0         2714       848           0        0
 sittingdown        0          832         0           0        0
 standing           0         3065       223           0        0
 standingup         0          828         0           0        0
 walking            0         1724      1210           0        0,
 'f1-score': 0.09218804613771409}

In [ ]:
list_of_f1_ann = []
list_of_c_matr_ann = []

In [ ]:
with tf.device('/CPU:0'):
  for i in range(len(ann_list)):
    print('train model ',i+1)
    print(ann_list[i])
    ann_list[i].fit(X_train2[index_sh ] , Y_tr_add[index_sh ]  , 1800 , 200 ,1 )
    print('estimation')
    Y_ann = ann_list[i].predict(X_val_mn)
    res_mod_i = conf_matr_and_score_f1(Y_ann ,Y_val , le)
    list_of_f1_ann.append( res_mod_i['f1-score'])
    list_of_c_matr_ann.append(res_mod_i['conf_matr'])
    

Выходные данные были обрезаны до нескольких последних строк (5000).
Epoch 1105/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.0541 - categorical_accuracy: 0.5621
Epoch 1106/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.0446 - categorical_accuracy: 0.5619
Epoch 1107/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.0596 - categorical_accuracy: 0.5598
Epoch 1108/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.0450 - categorical_accuracy: 0.5647
Epoch 1109/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.0503 - categorical_accuracy: 0.5641
Epoch 1110/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.0510 - categorical_accuracy: 0.5629
Epoch 1111/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.0451 - categorical_accuracy: 0.5593
Epoch 1112/1800
268/268 [==============================] - 1s 4ms/step - loss: 1.0488 - categorical_accuracy: 0.5

In [ ]:
np.argmax(list_of_f1_ann)

2

In [ ]:
list_of_f1_ann

NameError: ignored

In [ ]:
list_of_c_matr_ann[2]

,sitting,sittingdown,standing,standingup,walking
sitting,3529,31,2,0,0
sittingdown,4,550,13,265,0
standing,76,0,2201,761,250
standingup,0,42,33,753,0
walking,0,0,1346,227,1361


In [ ]:
ann_model =Keras_mlpclassifier(4 , 5 , [3 ] , [ 'sigmoid'  ] , SGD(learning_rate=0.007 ,momentum = 0.9 , nesterov = True) ,[ 'CategoricalAccuracy'] , 0.4,0.00001 )

In [ ]:
with tf.device('/CPU:0'):
  ann_model.fit(X_train2[index_sh ] , Y_tr_add[index_sh ]  , 600 , 200 ,1 )

NameError: ignored

Сохранение и загрузка моделей

In [ ]:
from joblib import dump, load

In [ ]:
dump(reslt_tree['best model'], '/content/drive/MyDrive/models_for_har4/tree.joblib')
dump(reslt_knn2['best model'], '/content/drive/MyDrive/models_for_har4/knn2.joblib')
dump(reslt_knn1['best model'], '/content/drive/MyDrive/models_for_har4/knn1.joblib')
dump(reslt_softmax['best model'], '/content/drive/MyDrive/models_for_har4/softmax.joblib')

['/content/drive/MyDrive/models_for_har4/softmax.joblib']

In [ ]:
tree = load( '/content/drive/MyDrive/models_for_har4/tree.joblib')
knn2 = load( '/content/drive/MyDrive/models_for_har4/knn2.joblib')
knn1 = load( '/content/drive/MyDrive/models_for_har4/knn1.joblib')
softmax  = load( '/content/drive/MyDrive/models_for_har4/softmax.joblib')

Оценка на тестовом наборе 

In [ ]:
result_of_knn1_test  = conf_matr_and_score_f1(reslt_knn1['best model'].predict(X_test_mn) ,Y_test , le)
result_of_knn2_test  = conf_matr_and_score_f1(reslt_knn2['best model'].predict(X_test_mn) ,Y_test , le)
result_of_tree_test  = conf_matr_and_score_f1(reslt_tree['best model'].predict(X_test_std) ,Y_test , le)
result_of_softmax_test  = conf_matr_and_score_f1(reslt_softmax['best model'].predict(X_test_std) ,Y_test , le)

Метод К соседей

In [ ]:
result_of_knn1_test['conf_matr'] 

,sitting,sittingdown,standing,standingup,walking
sitting,3562,0,0,0,0
sittingdown,3,805,2,16,6
standing,0,0,3265,0,24
standingup,0,2,0,826,0
walking,0,0,7,0,2926


In [ ]:
result_of_knn1_test['f1-score']


0.9947570779447745

Метод К соседей со взвешенной метрикой

In [ ]:
result_of_knn2_test['conf_matr'] 

,sitting,sittingdown,standing,standingup,walking
sitting,3561,0,1,0,0
sittingdown,2,813,2,10,5
standing,0,0,3276,3,10
standingup,0,1,4,823,0
walking,0,0,7,0,2926


In [ ]:
result_of_knn2_test['f1-score']

0.9960678084585809

Решающее дерево

In [ ]:
result_of_tree_test['conf_matr'] 

,sitting,sittingdown,standing,standingup,walking
sitting,3561,0,0,1,0
sittingdown,10,756,11,40,15
standing,0,13,3224,1,51
standingup,0,20,3,805,0
walking,0,9,53,1,2870


In [ ]:
result_of_tree_test['f1-score']

0.9800768961901433

Softmax - регрессия

In [ ]:
result_of_softmax_test['conf_matr'] 

,sitting,sittingdown,standing,standingup,walking
sitting,3508,53,0,1,0
sittingdown,46,615,106,46,19
standing,114,0,2608,110,457
standingup,1,133,223,463,8
walking,0,0,929,0,2004


In [ ]:
result_of_softmax_test['f1-score']

0.8037399510660608

Лучшая оценка у метода ближайших соседей со взвешенной метрикой

Оценка ансамблей



Импорт ансамблей

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from  sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from  sklearn.ensemble import VotingClassifier

In [ ]:
from sklearn.ensemble import StakingClassifier

In [ ]:
,5,7,8 , 10 , 12 ,14 ,16 ,18 , 20  , 22 ,24,26,28,30,35,40,45,50,55,60,70,80,90,100,120,130,140,150

Беггинг над методом ближайших соседей

In [ ]:
bag_knn2_params = {'n_estimators' : [2,3,4 ] , 
                    
                   'max_samples' : [ 0.75 , 0.8 , 0.85],
                     } 

In [ ]:
model_bag_knn2 = BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='wminkowski',
                      metric_params={'w': np.array([2.38, 2.86, 2.36, 0.73])},
                      n_jobs=-1, n_neighbors=2, p=2, weights='distance') ,
                      max_features=1.0 ,
                  bootstrap=True, 
                   random_state = 45,
                  oob_score=True, 
                    n_jobs= 20, verbose = 1) 

In [ ]:
reslt_bag_knn2 = fit_and_choose(X_train_mn,Y_train , 
                        model_bag_knn2  , 
                        bag_knn2_params ,
                       'f1_micro' , 
                        StratifiedKFold(n_splits=4,  shuffle=True, random_state = 42 ), 4,20  )

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   7 out of  36 | elapsed:  9.6min remaining: 39.7min
[Parallel(n_jobs=20)]: Done  17 out of  36 | elapsed: 14.1min remaining: 15.8min
[Parallel(n_jobs=20)]: Done  27 out of  36 | elapsed: 17.4min remaining:  5.8min
[Parallel(n_jobs=20)]: Done  36 out of  36 | elapsed: 21.3min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:633: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:638: RuntimeWarning: invalid value encountered in true_divid

In [ ]:
reslt_bag_knn2

{'best model': BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                       leaf_size=30,
                                                       metric='wminkowski',
                                                       metric_params={'w': array([2.38, 2.86, 2.36, 0.73])},
                                                       n_jobs=-1, n_neighbors=2,
                                                       p=2, weights='distance'),
                   bootstrap=True, bootstrap_features=False, max_features=1.0,
                   max_samples=0.85, n_estimators=4, n_jobs=20, oob_score=True,
                   random_state=45, verbose=1, warm_start=False),
 'best params': {'max_samples': 0.85, 'n_estimators': 4},
 'best score': 0.994669410814161}

Продолжение поиска

In [ ]:
model_bag_knn2_2 = BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                       leaf_size=30,
                                                       metric='wminkowski',
                                                       metric_params={'w': np.array([2.38, 2.86, 2.36, 0.73])},
                                                       n_jobs=-1, n_neighbors=2,
                                                       p=2, weights='distance'),
                   bootstrap=True, bootstrap_features=False, max_features=1.0,
                   max_samples=0.85,  n_jobs=20, oob_score=True,
                   random_state=45, verbose=1, warm_start=False)

In [ ]:
bag_knn2_params = {'n_estimators' :[5, 8 , 10 , 12 ,14 ,16 ,18 , 20,25,30 ,35 , 40 ]}
reslt_bag_knn2_2 = fit_and_choose(X_train_mn,Y_train , 
                        model_bag_knn2_2  , 
                        bag_knn2_params ,
                       'f1_micro' , 
                        StratifiedKFold(n_splits=4,  shuffle=True, random_state = 42 ), 4,20  )

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  22 out of  48 | elapsed: 61.2min remaining: 72.3min
[Parallel(n_jobs=20)]: Done  35 out of  48 | elapsed: 131.4min remaining: 48.8min
[Parallel(n_jobs=20)]: Done  48 out of  48 | elapsed: 195.6min remaining:    0.0s
[Parallel(n_jobs=20)]: Done  48 out of  48 | elapsed: 195.6min finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:   10.3s finished


Сохранение модели

In [ ]:
dump(reslt_bag_knn2_2['best model'], '/content/drive/MyDrive/models_for_har4/model_bag_knn2.joblib')
 

['/content/drive/MyDrive/models_for_har4/model_bag_knn2.joblib']

Оценка на валидационном наборе

In [ ]:
result_of_bag_knn2  = conf_matr_and_score_f1(reslt_bag_knn2_2['best model'].predict(X_val_mn) ,Y_val , le)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:  1.5min remaining: 13.7min
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:  4.7min finished


In [ ]:
reslt_bag_knn2_2['best model']

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='wminkowski',
                                                      metric_params={'w': array([2.38, 2.86, 2.36, 0.73])},
                                                      n_jobs=-1, n_neighbors=2,
                                                      p=2, weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=0.85, n_estimators=30, n_jobs=20, oob_score=True,
                  random_state=45, verbose=1, warm_start=False)

In [ ]:
result_of_bag_knn2

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3561            0         1           0        0
 sittingdown        3          813         3           9        4
 standing           0            0      3280           1        7
 standingup         0            0         3         825        0
 walking            0            0        16           0     2918,
 'f1-score': 0.9958930443900734}

Оценка на тестовом наборе

In [ ]:
result_of_bag_knn2_2_test  = conf_matr_and_score_f1(reslt_bag_knn2_2['best model'].predict(X_test_mn) ,Y_test , le)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   2 out of  20 | elapsed:  1.4min remaining: 12.9min
[Parallel(n_jobs=20)]: Done  20 out of  20 | elapsed:  4.5min finished


In [ ]:
result_of_bag_knn2_2_test

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3560            0         2           0        0
 sittingdown        3          808         2          14        5
 standing           0            0      3274           3       12
 standingup         0            1         5         822        0
 walking            0            0        13           0     2920,
 'f1-score': 0.9947570779447745}

Оценка случайного леса

Параметры подбираются случайным поиском по решетке

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf_pars = {'n_estimators': [5,10,15,20,25,30,35,40],
           'criterion': ['gini' , 'entropy'], 
           'max_depth' : [4,6,8,10,12]  , 
           'min_samples_leaf' : [2 , 4 , 6 , 8 , 10 , 12 , 16 , 18 , 20] , 
           'min_samples_split' : [ 2 , 4 , 6 , 8 , 10 , 12 , 16 , 18 , 20] , 
           'max_samples' : [0.7 , 0.85]}
rf_model = RandomForestClassifier( random_state = 67 , verbose =1, n_jobs = 20 ,bootstrap = True )

In [ ]:
rf_pars = {'n_estimators': [5,10,15,20,25,30,35,40],
           'criterion': ['gini' , 'entropy'], 
           'max_depth' : [4,6,8,10,12]  , 
           'min_samples_leaf' : [2 , 4 , 6 , 8 , 10 , 12 , 16 , 18 , 20] , 
           'min_samples_split' : [ 2 , 4 , 6 , 8 , 10 , 12 , 16 , 18 , 20] , 
           'max_samples' : [0.7 , 0.85]}
for i in rf_pars:
  print(i,': ',sep='',end='')
  for p in rf_pars[i]:
    print( p,', ',end='',sep='')
  print()

n_estimators: 5, 10, 15, 20, 25, 30, 35, 40, 
criterion: gini, entropy, 
max_depth: 4, 6, 8, 10, 12, 
min_samples_leaf: 2, 4, 6, 8, 10, 12, 16, 18, 20, 
min_samples_split: 2, 4, 6, 8, 10, 12, 16, 18, 20, 
max_samples: 0.7, 0.85, 


In [ ]:
'''reslt_rf = fit_and_choose(X_train_std ,Y_train , 
                        rf_model  , 
                        rf_pars ,
                       'f1_micro' , 
                        StratifiedKFold(n_splits=4,  shuffle=True, random_state = 42 ), 4,20  )'''
rgs_rf = RandomizedSearchCV(estimator = rf_model , 
                   param_distributions = rf_pars , 
                   n_iter = 2000 , 
                  scoring =  'f1_micro',
                   n_jobs = 20  ,
                   refit = True  , 
                   cv =  StratifiedKFold(n_splits=4,  shuffle=True, random_state = 42 ),
                   verbose = 4,
                   random_state = 67)
reslt_rf = rgs_rf.fit(X_train_std ,Y_train) 

Fitting 4 folds for each of 2000 candidates, totalling 8000 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:  1.1min
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:  2.6min
[Parallel(n_jobs=20)]: Done 352 tasks      | elapsed:  4.4min
[Parallel(n_jobs=20)]: Done 573 tasks      | elapsed:  7.7min
[Parallel(n_jobs=20)]: Done 842 tasks      | elapsed: 11.7min
[Parallel(n_jobs=20)]: Done 1161 tasks      | elapsed: 15.5min
[Parallel(n_jobs=20)]: Done 1528 tasks      | elapsed: 20.5min
[Parallel(n_jobs=20)]: Done 1945 tasks      | elapsed: 25.1min
[Parallel(n_jobs=20)]: Done 2410 tasks      | elapsed: 31.4min
[Parallel(n_jobs=20)]: Done 2925 tasks      | elapsed: 37.8min
[Parallel(n_jobs=20)]: Done 3488 tasks      | elapsed: 44.2min
[Parallel(n_jobs=20)]: Done 4101 tasks      | elapsed: 52.0min
[Parallel(n_jobs=20)]: Done 4762 tasks      | elapsed: 60.5min
[Parallel(n_jobs=20)]: Done 5473 tasks      | elapsed: 70.7min
[Parallel(n_jobs=20)]: Done 6232 tasks      | 

In [ ]:
rforest = reslt_rf.best_estimator_

In [ ]:
rforest 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features='auto',
                       max_leaf_nodes=None, max_samples=0.85,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=35, n_jobs=20,
                       oob_score=False, random_state=67, verbose=1,
                       warm_start=False)

In [ ]:
result_of_rforest  = conf_matr_and_score_f1(rforest.predict(X_val_std) ,Y_val , le)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  32 out of  35 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  35 out of  35 | elapsed:    0.0s finished


In [ ]:
result_of_rforest

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3562            0         0           0        0
 sittingdown        2          819         3           4        4
 standing           0            0      3263           0       25
 standingup         0            4         2         822        0
 walking            0            0        24           0     2910,
 'f1-score': 0.9940580216707445}

In [ ]:
rforest2 = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features='auto',
                       max_leaf_nodes=None, max_samples=0.85,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=20,
                       oob_score=False, random_state=67, verbose=1,
                       warm_start=False)

In [ ]:
rforest2.fit(X_train_std ,Y_train) 

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    6.7s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   16.6s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   23.6s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features='auto',
                       max_leaf_nodes=None, max_samples=0.85,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=20, oob_score=False, random_state=67, verbose=1,
                       warm_start=False)

In [ ]:
result_of_rforest2  = conf_matr_and_score_f1(rforest2.predict(X_val_std) ,Y_val , le)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.6s finished


In [ ]:
result_of_rforest2  

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3562            0         0           0        0
 sittingdown        2          818         3           5        4
 standing           0            0      3261           0       27
 standingup         0            3         0         825        0
 walking            0            0        22           0     2912,
 'f1-score': 0.994232785739252}

 Проверка на тестовой выборке
 

In [ ]:
result_of_rforest2_test  = conf_matr_and_score_f1(rforest2.predict(X_test_std) ,Y_test , le)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.6s finished


In [ ]:
result_of_rforest2_test 

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3562            0         0           0        0
 sittingdown        3          808         2          11        8
 standing           0            0      3262           0       27
 standingup         0            2         1         825        0
 walking            0            0        25           0     2908,
 'f1-score': 0.9930968192939532}

Случайный лес справился хуже чем

Сохранение модели

In [ ]:
dump(rforest2, '/content/drive/MyDrive/models_for_har4/model_rforest.joblib')
 

['/content/drive/MyDrive/models_for_har4/model_rforest.joblib']

Рассмотрим алгоритм градиентного бустинга
(GradientBoostingClassifier)

In [ ]:
from  sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model_grboost =  GradientBoostingClassifier(loss = 'deviance' , 
                                             criterion = 'friedman_mse' ,
                                              random_state = 87, n_estimators = 20 , learning_rate = 0.4  ,                                         
                                             min_impurity_split = None , verbose = 4   , warm_start=True )  
                                          
                    

In [ ]:
grboost_params1 = {        
                          
                   'max_depth' : [2, 3, 5 ],
                  'min_samples_split' : [20 , 30 , 40 , 50 , 60    ] , 
                  'min_samples_leaf' : [3 , 8 ,10 , 20 , 25    ]
                   }

In [ ]:
rf_pars  = {        
                          
                   'max_depth' : [2, 3, 5 ],
                  'min_samples_split' : [20 , 30 , 40 , 50 , 60    ] , 
                  'min_samples_leaf' : [3 , 8 ,10 , 20 , 25    ]
                   }
 
for i in rf_pars:
  print(i,': ',sep='',end='')
  for p in rf_pars[i]:
    print( p,', ',end='',sep='')
  print()

max_depth: 2, 3, 5, 
min_samples_split: 20, 30, 40, 50, 60, 
min_samples_leaf: 3, 8, 10, 20, 25, 


In [ ]:
 'n_estimators' : [ 10 , 20 , 30 , 40 , 50 , 60 , 70 , 80 , 90 , 100 ,200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 ] 
 'learning_rate' :[  0.08 ,0.1    , 0.4 ,   1.0],
 , 40 , 45 , 50 , 55 , 60
 , 90 , 100 , 110 , 120 , 130 , 140 , 150 

20 , 25 , 30 , 35 , 40 , 45 , 50 , 55 , 60 , 

In [ ]:
 result_of_grboost1 = fit_and_choose( X_train_std , Y_train ,
                                     model_grboost ,
                                     grboost_params1 , 'f1_micro' , 
                                      StratifiedKFold(n_splits=4,  shuffle=True, random_state = 42 ), 
                                     5, 20)

Fitting 4 folds for each of 75 candidates, totalling 300 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:  2.5min
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:  7.7min
[Parallel(n_jobs=20)]: Done 248 tasks      | elapsed: 19.8min
[Parallel(n_jobs=20)]: Done 300 out of 300 | elapsed: 24.1min finished


      Iter       Train Loss   Remaining Time 
         1       15764.9704            9.82s
         2        8648.2091           10.00s
         3        5151.7936            9.56s
         4        3137.0128            9.01s
         5        2102.6541            8.51s
         6        1441.1314            7.98s
         7        1060.4248            7.40s
         8         819.5300            6.82s
         9         702.8182            6.27s
        10         546.8455            5.71s
        11         469.0541            5.14s
        12         428.1314            4.57s
        13         369.6172            4.01s
        14         321.4938            3.45s
        15         281.5701            2.87s
        16         255.9806            2.29s
        17         229.7847            1.72s
        18         212.9240            1.14s
        19         199.3673            0.57s
        20         182.7220            0.00s


In [ ]:
 result_of_grboost1

{'best model': GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.4, loss='deviance', max_depth=5,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=20, min_samples_split=50,
                            min_weight_fraction_leaf=0.0, n_estimators=20,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=87, subsample=1.0, tol=0.0001,
                            validation_fraction=0.1, verbose=4, warm_start=True),
 'best params': {'max_depth': 5,
  'min_samples_leaf': 20,
  'min_samples_split': 50},
 'best score': 0.9946402766814013}

In [ ]:
 grboost_params2 = {        
                          
                   'n_estimators' : [  20 , 30 , 40 , 50 , 60 , 70 , 80 , 90   ] ,
                 'learning_rate' :[  0.08 ,0.1    , 0.4 ,   1.0], 
                   
                   }

In [ ]:
, 100 ,200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500

In [ ]:
model_grboost2 = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                              loss='deviance', max_depth=5,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=20, min_samples_split=50,
                            min_weight_fraction_leaf=0.0,  
                            n_iter_no_change=None, presort='deprecated',
                            random_state=87, subsample=1.0, tol=0.0001,
                            validation_fraction=0.1, verbose=4, warm_start=True)

In [ ]:
 result_of_grboost2 = fit_and_choose( X_train_std , Y_train ,
                                     model_grboost2 ,
                                     grboost_params2 , 'f1_micro' , 
                                      StratifiedKFold(n_splits=4,  shuffle=True, random_state = 42 ), 
                                     5, 20)

Fitting 4 folds for each of 32 candidates, totalling 128 fits


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed: 12.0min
[Parallel(n_jobs=20)]: Done 115 out of 128 | elapsed: 38.3min remaining:  4.3min
[Parallel(n_jobs=20)]: Done 128 out of 128 | elapsed: 40.9min finished


      Iter       Train Loss   Remaining Time 
         1       15764.9704           40.88s
         2        8648.2091           43.03s
         3        5151.7936           43.23s
         4        3137.0128           42.93s
         5        2102.6541           42.61s
         6        1441.1314           42.03s
         7        1060.4248           41.49s
         8         819.5300           40.77s
         9         702.8182           40.22s
        10         546.8455           39.72s
        11         469.0541           39.26s
        12         428.1314           38.71s
        13         369.6172           38.22s
        14         321.4938           37.68s
        15         281.5701           37.16s
        16         255.9806           36.59s
        17         229.7847           36.04s
        18         212.9240           35.38s
        19         199.3673           34.73s
        20         182.7220           34.07s
        21         167.3591           33.48s
        2

In [ ]:
from joblib import dump, load
dump(result_of_grboost2['best model'] , '/content/drive/MyDrive/models_for_har4/model_grad_boost_tree.joblib' )

NameError: ignored

In [ ]:
resu_of_grboost2  = conf_matr_and_score_f1(result_of_grboost2['best model'].predict(X_val_std) ,Y_val , le)

In [ ]:
result_of_grboost2['best model']

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.4, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=20, min_samples_split=50,
                           min_weight_fraction_leaf=0.0, n_estimators=80,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=87, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=4, warm_start=True)

In [ ]:
resu_of_grboost2 

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3560            2         0           0        0
 sittingdown        3          819         3           4        3
 standing           0            0      3272           0       16
 standingup         0            0         0         828        0
 walking            0            0        24           0     2910,
 'f1-score': 0.9951939881160433}

In [ ]:
model_grboost3 = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.4, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=20, min_samples_split=50,
                           min_weight_fraction_leaf=0.0, n_estimators=135,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=87, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=4, warm_start=True)

In [ ]:
model_grboost3.fit(X_train_std  ,Y_train)

      Iter       Train Loss   Remaining Time 
         1       15764.9704            1.12m
         2        8648.2091            1.21m
         3        5151.7936            1.23m
         4        3137.0128            1.23m
         5        2102.6541            1.23m
         6        1441.1314            1.22m
         7        1060.4248            1.21m
         8         819.5300            1.19m
         9         702.8182            1.19m
        10         546.8455            1.18m
        11         469.0541            1.17m
        12         428.1314            1.16m
        13         369.6172            1.16m
        14         321.4938            1.15m
        15         281.5701            1.14m
        16         255.9806            1.13m
        17         229.7847            1.12m
        18         212.9240            1.11m
        19         199.3673            1.10m
        20         182.7220            1.08m
        21         167.3591            1.07m
        2

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.4, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=20, min_samples_split=50,
                           min_weight_fraction_leaf=0.0, n_estimators=135,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=87, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=4, warm_start=True)

In [ ]:
resu_of_grboost3  = conf_matr_and_score_f1(model_grboost3.predict(X_val_std) ,Y_val , le)

In [ ]:
resu_of_grboost3 

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3560            2         0           0        0
 sittingdown        3          819         3           4        3
 standing           0            0      3275           0       13
 standingup         0            0         0         828        0
 walking            0            0        23           0     2911,
 'f1-score': 0.9955435162530584}

In [ ]:
resu_of_grboost3_test  = conf_matr_and_score_f1(model_grboost3.predict(X_test_std) ,Y_test , le)

In [ ]:
resu_of_grboost3_test 

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3560            2         0           0        0
 sittingdown        2          808         1          13        8
 standing           0            0      3273           0       16
 standingup         0            0         0         828        0
 walking            0            0        20           0     2913,
 'f1-score': 0.994582313876267}

In [ ]:
from joblib import dump, load
dump(model_grboost3 , '/content/drive/MyDrive/models_for_har4/model_grad_boost_tree.joblib')
 

['/content/drive/MyDrive/models_for_har4/model_grad_boost_tree.joblib']

Гдадиентный бустинг над деревьями по качеству уступает  

  Оценим теперь несколько модели, соединенных в 1. 
Голосующий классификатор 
Голосование простое

In [ ]:
from sklearn.ensemble import VotingClassifier

Загрузка моделей 
Отдельно помечено - какой масштаб данных нужен для моделей

In [ ]:
from joblib import dump, load

In [ ]:
#standart data
gboost = load(  '/content/drive/MyDrive/models_for_har4/model_grad_boost_tree.joblib')
rforest = load('/content/drive/MyDrive/models_for_har4/model_rforest.joblib')
tree = load( '/content/drive/MyDrive/models_for_har4/tree.joblib')
softmax = load( '/content/drive/MyDrive/models_for_har4/softmax.joblib')
#min max data
knn1 = load( '/content/drive/MyDrive/models_for_har4/knn1.joblib')
bag_knn2 = load( '/content/drive/MyDrive/models_for_har4/model_bag_knn2_40.joblib') 
knn2 = load( '/content/drive/MyDrive/models_for_har4/knn2.joblib')

In [ ]:
bag_knn2

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='wminkowski',
                                                      metric_params={'w': array([2.38, 2.86, 2.36, 0.73])},
                                                      n_jobs=-1, n_neighbors=2,
                                                      p=2, weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=0.85, n_estimators=40, n_jobs=-1, oob_score=False,
                  random_state=45, verbose=1, warm_start=False)

In [ ]:
conf_matr_and_score_f1(bag_knn2.predict(X_test_mn) ,Y_test , le)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  5.9min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  5.9min finished


{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3560            0         2           0        0
 sittingdown        3          806         3          15        5
 standing           0            0      3275           3       11
 standingup         0            1         7         820        0
 walking            0            0        12           0     2921,
 'f1-score': 0.994582313876267}

In [ ]:
estm_list = [ 
             
             ('rforest' , rforest  ),
             ('gboost' , gboost )]

In [ ]:
vc1 = VotingClassifier(estimators = estm_list  , voting='hard')

In [ ]:
vc1.fit(X_train_std , Y_train)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.8s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    8.0s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:   20.0s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:   28.7s finished


      Iter       Train Loss   Remaining Time 
         1       15764.9704            1.17m
         2        8648.2091            1.24m
         3        5151.7936            1.24m
         4        3137.0128            1.24m
         5        2102.6541            1.23m
         6        1441.1314            1.23m
         7        1060.4248            1.22m
         8         819.5300            1.21m
         9         702.8182            1.21m
        10         546.8455            1.20m
        11         469.0541            1.19m
        12         428.1314            1.18m
        13         369.6172            1.17m
        14         321.4938            1.16m
        15         281.5701            1.15m
        16         255.9806            1.14m
        17         229.7847            1.13m
        18         212.9240            1.12m
        19         199.3673            1.11m
        20         182.7220            1.10m
        21         167.3591            1.08m
        2

VotingClassifier(estimators=[('rforest',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=12,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=0.85,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=2,
                                                     min_samples_split=8,
                                                     min_weight_fraction_leaf=0.0,
     

In [ ]:
result_of_vc1_val  = conf_matr_and_score_f1(vc1.predict(X_val_std) ,Y_val , le)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.7s finished


In [ ]:
result_of_vc1_val 

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3562            0         0           0        0
 sittingdown        3          819         3           5        2
 standing           0            0      3279           0        9
 standingup         0            3         0         825        0
 walking            0            0        29           0     2905,
 'f1-score': 0.995281370150297}

In [ ]:
dump(vc1 ,'/content/drive/MyDrive/models_for_har4/model_vote_gboost_rforest.joblib')

['/content/drive/MyDrive/models_for_har4/model_vote_gboost_rforest.joblib']

Голосование между 
бустингом и лесом показало высокий результат (выше чем отдельный алгоритм)

In [ ]:
estm_list_mn = [ 
             
             ('knn2' , knn2  ),
             ('bag_knn2' , bag_knn2 )]
vc_mn = VotingClassifier(estimators = estm_list_mn , voting='hard')


In [ ]:
vc_mn.fit(X_train_mn , Y_train)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.7s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.7s finished


VotingClassifier(estimators=[('knn2',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='wminkowski',
                                                   metric_params={'w': array([2.38, 2.86, 2.36, 0.73])},
                                                   n_jobs=None, n_neighbors=2,
                                                   p=2, weights='distance')),
                             ('bag_knn2',
                              BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                                                    leaf_size=30,
                                                                                    metric='wminkowski',
                                                                                    metric_params={'w': array([2.38, 2.86, 2.36, 0.73])},
         

In [ ]:
result_of_vc1_mn  = conf_matr_and_score_f1(vc_mn.predict(X_val_mn) ,Y_val , le)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  5.9min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  5.9min finished


In [ ]:
result_of_vc1_mn

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3561            0         1           0        0
 sittingdown        3          816         3           6        4
 standing           0            0      3282           1        5
 standingup         0            0         3         825        0
 walking            0            0        17           0     2917,
 'f1-score': 0.9962425725270885}

Голосующий классификатор из

метода соседей и

беггинга над методом соседей (у обоих взвешенные метрики)
показал результат чуть лучше чем 1 классификатор

In [ ]:
estm_list_mn2 = [ 
             ('knn1' , knn1  ),
             ('knn2' , knn2  ),
             ('bag_knn2' , bag_knn2 )]
vc_mn2 = VotingClassifier(estimators = estm_list_mn2 , voting='hard')


In [ ]:
vc_mn2.fit(X_train_mn , Y_train)
result_of_vc1_mn2  = conf_matr_and_score_f1(vc_mn2.predict(X_val_mn) ,Y_val , le)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    3.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  6.0min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  6.0min finished


In [ ]:
result_of_vc1_mn2  = conf_matr_and_score_f1(vc_mn2.predict(X_val_mn) ,Y_val , le)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  6.0min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  6.0min finished


In [ ]:
result_of_vc1_mn2

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3561            0         1           0        0
 sittingdown        3          813         3           9        4
 standing           0            0      3281           1        6
 standingup         0            0         1         827        0
 walking            0            0        15           0     2919,
 'f1-score': 0.9962425725270885}

Качество значительно не улучшилось

Загрузка всех сохраненных моделей и получение таблицы с оценками на тестовой выборке

In [ ]:
 
dump(vc_mn ,'/content/drive/MyDrive/models_for_har4/model_vote_knn2_bag-knn2.joblib')

['/content/drive/MyDrive/models_for_har4/model_vote_knn2_bag-knn2.joblib']

In [ ]:
gboost = load(  '/content/drive/MyDrive/models_for_har4/model_grad_boost_tree.joblib')
rforest = load('/content/drive/MyDrive/models_for_har4/model_rforest.joblib')
tree = load( '/content/drive/MyDrive/models_for_har4/tree.joblib')
softmax = load( '/content/drive/MyDrive/models_for_har4/softmax.joblib')
#min max data
knn1 = load( '/content/drive/MyDrive/models_for_har4/knn1.joblib')
bag_knn2 = load( '/content/drive/MyDrive/models_for_har4/model_bag_knn2_40.joblib') 
knn2 = load( '/content/drive/MyDrive/models_for_har4/knn2.joblib')
st_models = [softmax ,tree , rforest,gboost  ]
mn_models = [knn1 , knn2 ,bag_knn2   ]

In [ ]:
conf_matr_and_score_f1(gboost.predict(X_test_std) ,Y_test , le)

{'conf_matr':              sitting  sittingdown  standing  standingup  walking
 sitting         3560            2         0           0        0
 sittingdown        2          808         1          13        8
 standing           0            0      3273           0       16
 standingup         0            0         0         828        0
 walking            0            0        20           0     2913,
 'f1-score': 0.994582313876267}

In [ ]:
model_names1 = ['softmax regression',
               'decision tree',
                
               'random forest',
               'gradient boosting - tree']
model_names2 = ['K nearest neighbors',
               'K nearest neighbors (weghted metric)',
               'bagging - K nearest neighbors (weghted metric)']
model_names3 =  [              
               'vote classifier - random forest, gradient boosting - tree',
               'vote classifier - bagging - K nearest neighbors (weghted metric), K nearest neighbors (weghted metric)']

In [ ]:
conf_mat = []
f1_scores = []
t = 0
for i in st_models:
  print(model_names1[t])
  t+=1
  reslts  = conf_matr_and_score_f1(i.predict(X_test_std ) , Y_test,le)
  conf_mat.append(reslts['conf_matr'])
  f1_scores.append(reslts['f1-score'])
t =0
for i in mn_models:
  print(model_names2[t])
  t+=1
  reslts  = conf_matr_and_score_f1(i.predict(X_test_mn ) , Y_test,le)
  conf_mat.append(reslts['conf_matr'])
  f1_scores.append(reslts['f1-score'])

print(model_names3[0])
reslts  = conf_matr_and_score_f1(vc1.predict(X_test_std ) , Y_test,le)
conf_mat.append(reslts['conf_matr'])
f1_scores.append(reslts['f1-score'])

print(model_names3[1])
reslts  = conf_matr_and_score_f1(vc_mn.predict(X_test_mn ) , Y_test,le)
conf_mat.append(reslts['conf_matr'])
f1_scores.append(reslts['f1-score'])
names_models = model_names1 + model_names2 + model_names3
table_of_f1 = {names_models[i] : f1_scores[i] for  i in range(len( f1_scores) )}

softmax regression
decision tree
random forest


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.7s finished


gradient boosting - tree
K nearest neighbors
K nearest neighbors (weghted metric)
bagging - K nearest neighbors (weghted metric)


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  6.0min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  6.0min finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.0s


vote classifier - random forest, gradient boosting - tree


[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 410 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 600 out of 600 | elapsed:    0.7s finished


vote classifier - bagging - K nearest neighbors (weghted metric), K nearest neighbors (weghted metric)


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  6.0min remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  6.0min finished


In [ ]:
table_scores = pd.DataFrame(data = table_of_f1.items() ,columns = ['model' , 'f1 micro'])

In [ ]:
table_scores = table_scores.sort_values(by ='f1 micro')

In [ ]:
table_scores.to_csv('/content/drive/MyDrive/данные с акселерометра/оценки для 4.csv', sep = ';')